In [ ]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""


In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.194
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## CoT - Chain of thought Reasoning

In [ ]:
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(model_name='text-davinci-003',
             temperature=0,
             max_tokens = 256)

In [ ]:
text = "Explain step by step. How old is the president of the United States?"

print(llm(text))



1. The current President of the United States is Joe Biden. 

2. Joe Biden was born on November 20, 1942. 

3. To calculate his age, subtract his birth year (1942) from the current year (2021). 

4. Therefore, Joe Biden is 78 years old.


In [ ]:
text = "How would I get from Singapore to San Francisco?"

print(llm(text))



The most direct route from Singapore to San Francisco is to fly. There are several airlines that offer direct flights from Singapore to San Francisco, including Singapore Airlines, United Airlines, and American Airlines. The flight time is approximately 15 hours.


In [ ]:
text = "Explain step by step. How would I get from Singapore to San Francisco?"

print(llm(text))



1. Book a flight from Singapore to San Francisco. You can do this online or through a travel agent.

2. Pack your bags and make sure you have all the necessary documents for travel, such as a passport and visa if necessary.

3. Arrive at the airport in Singapore at least two hours before your flight.

4. Check in for your flight and go through security.

5. Board your flight and enjoy the journey.

6. Arrive at San Francisco International Airport.

7. Go through customs and immigration.

8. Collect your luggage and make your way to your destination.


## Manual ReACT


In [ ]:
question = "How old is the president of the United States?"

manual_react = f"""Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]
Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.
Action: Search[Milhouse]
Observation: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.
Thought: The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".
Action: Lookup[named after]
Observation: (Result 1 / 1) Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.
Thought: Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Thought: I need to search Adam Clayton Powell and The Saimaa Gesture, and find which documentary is about Finnish rock groups.
Action: Search[Adam Clayton Powell]
Observation: Could not find [Adam Clayton Powell]. Similar: [’Adam Clayton Powell III’, ’Seventh Avenue (Manhattan)’, ’Adam Clayton Powell Jr. State Office Building’, ’Isabel Washington Powell’, ’Adam Powell’, ’Adam Clayton Powell (film)’, ’Giancarlo Esposito’].
Thought: To find the documentary, I can search Adam Clayton Powell (film).
Action: Search[Adam Clayton Powell (film)]
Observation: Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg. The film is about the rise and fall of influential African-American politician Adam Clayton Powell Jr.[3][4] It was later aired as part of the PBS series The American Experience.
Thought: Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.
Action: Finish[The Saimaa Gesture]

Question: What profession does Nicholas Ray and Elia Kazan have in common?
Thought: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.
Action: Search[Nicholas Ray]
Observation: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.
Thought: Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.
Action: Search[Elia Kazan]
Observation: Elia Kazan was an American film and theatre director, producer, screenwriter and actor.
Thought: Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.
Action: Finish[director, screenwriter, actor]

Question: Which magazine was started first Arthur’s Magazine or First for Women?
Thought: I need to search Arthur’s Magazine and First for Women, and find which was started first.
Action: Search[Arthur’s Magazine]
Observation: Arthur’s Magazine (1844-1846) was an American literary periodical published in Philadelphia in the 19th century.
Thought: Arthur’s Magazine was started in 1844. I need to search First for Women next.
Action: Search[First for Women]
Observation: First for Women is a woman’s magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989.
Thought: First for Women was started in 1989. 1844 (Arthur’s Magazine) < 1989 (First for Women), so Arthur’s Magazine was started first.
Action: Finish[Arthur’s Magazine]

Question:{question}"""

In [ ]:
print(llm(manual_react))


Thought: I need to search the president of the United States, find their age, and answer the question.
Action: Search[president of the United States]
Observation: Joe Biden is the 46th and current president of the United States.
Thought: Joe Biden is the president of the United States. I need to search Joe Biden and find his age.
Action: Search[Joe Biden]
Observation: Joseph Robinette Biden Jr. (born November 20, 1942) is an American politician who is the 46th and current president of the United States.
Thought: Joe Biden was born in 1942, so he is 78 years old.
Action: Finish[78 years old]


## ReACT

“REACT: SYNERGIZING REASONING AND ACTING IN LANGUAGE MODELS” (Shunyu et al., 2022)

In [ ]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

In [ ]:

docstore=DocstoreExplorer(Wikipedia())

tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

llm = OpenAI(temperature=0, model_name="text-davinci-003")


react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [ ]:
question = "How old is the president of the United States?"
react.run(question)



> Entering new AgentExecutor chain...

Thought: I need to search the president of the United States, find his age, and then finish.

Action: Search[president of the United States]

Observation: The President of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, with notable expansion during the presidency of Franklin D. Roosevelt. In modern times, the president is one of the world's most powerful political figures – the leader of the only remaining global superpower. As the leader of the nation with the 

'78 years old'

In [ ]:
question = "How old will the president of the United States be in June 2023?"
react.run(question)



> Entering new AgentExecutor chain...

Thought: I need to search the president of the United States, find their age, then find how old they will be in June 2023.

Action: Search[president of the United States]

Observation: The President of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, with notable expansion during the presidency of Franklin D. Roosevelt. In modern times, the president is one of the world's most powerful political figures – the leader of the only remaining global superpower. As the 

'80 years old'

In [ ]:
question = "How old is the president of the United States?"
react.run(question)

#### Let pull apart the Chain

In [ ]:
print(react.agent.llm_chain.prompt.template)
the



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United St

In [ ]:
# multi hop example
question = "What is the first film that Russell Crowe won an Oscar for, and who directed that movie?"
react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search Russell Crowe and find the first film he won an Oscar for, and who directed that movie.
Action: Search[Russell Crowe]
Observation: Could not find [Russell Crowe]. Similar: ['Russell Crowe', 'Russell Crowe filmography', "The Pope's Exorcist", 'Unhinged (2020 film)', 'Robin Hood (2010 film)', "Russell Crowe's jockstrap", 'Dave Crowe', 'Sleeping Dogs (upcoming film)', 'List of awards and nominations received by Russell Crowe', 'Gladiator (2000 film)']
Thought: To find the first film Russell Crowe won an Oscar for, I can search Gladiator (2000 film).
Action: Search[Gladiator (2000 film)]
Observation: Gladiator is a 2000 epic historical drama film directed by Ridley Scott and written by David Franzoni, John Logan, and William Nicholson. It was produced by Universal Pictures. DreamWorks Pictures distributed the film in North America while Universal Pictures released it internationally through United International Pictures. It 

'Gladiator, Ridley Scott'

In [ ]:
import langchain

langchain.debug = True

In [ ]:
# multi hop example
question = "What is the first film that Russell Crowe won an Oscar for, and who directed that movie?"
react.run(question)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the first film that Russell Crowe won an Oscar for, and who directed that movie?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is the first film that Russell Crowe won an Oscar for, and who directed that movie?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nThought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.\nAction: Search[Colorado orogeny]\nObservation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.\nThought: It doe

'Gladiator (2000 film), Ridley Scott'

In [ ]:
langchain.debug = False